<a href="https://colab.research.google.com/github/afontoura/DRLA2ALM/blob/master/stable_baselines_DDPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install stable_baselines==2.9.0
!git clone https://github.com/afontoura/DRLA2ALM
%cd DRLA2ALM
!pip install -e .

In [0]:
import gym
import envs
import numpy as np

from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines import DDPG

time_horizon = 80
discount_rate = .08
current_seed = 10

env = gym.make('ALM-v0', T = time_horizon, rate = discount_rate)

In [0]:
# the noise objects for DDPG
n_actions = env.action_space.shape[-1]
param_noise = None
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

model = DDPG(MlpPolicy, env, verbose=1, param_noise=param_noise, action_noise=action_noise, seed = current_seed)
model.learn(total_timesteps=500000)

In [0]:
resultados = []
for i in np.arange(1000):
  dones = False
  G = []
  obs = env.reset()
  while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    G.append(rewards)
  resultados.append(np.array(G).sum())
  #print(str(i) + ': ' + str(np.array(G).sum()))

In [0]:
print((np.array(resultados) == time_horizon).sum())
print(np.array(resultados).mean())

In [0]:
model.save('ddpg_t' + str(time_horizon) + '_dr' + str(round(discount_rate * 1000)) + '_seed' + str(current_seed))